# Workflow 3: comparing observations to emissions

In addition to observation files, ancillary data can also be added to an openghg object store which can be used to perform analysis.

At the moment, the accepted files include:
 - Footprints - regional outputs from an LPDM model (e.g. NAME)
 - Emissions/Flux - estimates of species emissions within a region
 - *[+Boundary conditions - to be added+]*
 - Global CTM output (e.g. GEOSChem)

These inputs must adhere to an expected format and are expected to minimally contain a fixed set of inputs.

*At the moment, the expected format for these files is created through standard methods from within the ACRG repository.*

> **_NOTE:_**  Plots created within this tutorial may not show up on the online documentation version of this notebook.

## 1. Loading data sources into the object store

For this tutorial we will again set up a temporary object store to store our data.

See [1_Adding_observation_data.ipynb](1_Adding_observation_data.ipynb) for more details and advice on how to create a more permanent object store. Once a permanent object store is set up, these steps would only need to be performed once. Any added data can then be retrieved using searches.

In [ ]:
import os
import tempfile

tmp_dir = tempfile.TemporaryDirectory()
os.environ["OPENGHG_PATH"] = tmp_dir.name   # temporary directory

%load_ext autoreload
%autoreload 2

For this, we will add observation, footprint and flux data to the object store. This data relates to Tacolneston (TAC) site within the DECC network and the area around Europe (EUROPE domain).

In [1]:
from openghg.util import retrieve_example_data
from openghg.standardise import standardise_surface

obs_data = retrieve_example_data(path="timeseries/tac_example.tar.gz")

site="tac" 
network="DECC"
height="100m"
species="ch4"

obs_results = standardise_surface(filepaths=obs_data, source_format="CRDS", site=site, network=network)

  0%| | 0/3

  0%| | 0/3


In [3]:
from openghg.standardise import standardise_flux

flux_data = retrieve_example_data(path="flux/ch4-ukghg-all_EUROPE_2016.tar.gz")
site="tac" 
domain="EUROPE"
date = "2016"
source_waste = "waste"
source_energyprod = "energyprod"
flux_data_waste = [filename for filename in flux_data if source_waste in str(filename)][0]
flux_data_energyprod = [filename for filename in flux_data if source_energyprod in str(filename)][0]


In [4]:
from openghg.standardise import standardise_footprint

footprint_data = retrieve_example_data(path="footprint/tac_footprint_inert_201607.tar.gz")

site="tac" 
height="100m"
domain="EUROPE"
model="NAME"

standardise_footprint(filepath=footprint_data, site=site, height=height, domain=domain, model=model)

## 2. Creating a model scenario

With this ancillary data, we can start to make comparisons between model data, such as bottom-up inventories, and our observations. This analysis is based around a `ModelScenario` class which can be created to link together observation, footprint and emissions data.

*Boundary conditions and other model data will be added soon*

Above we loaded observation data from the Tacolneston site into the object store. We also added an associated footprint (sensitivity map) and anthropogenic emissions maps both for a domain defined over Europe.

To access and link this data we can set up our `ModelScenario` instance using a similiar set of keywords. In this case we have also limited ourselves to a date range:

In [5]:
from openghg.analyse import ModelScenario

start_date = "2016-07-01"
end_date = "2016-08-01"

scenario = ModelScenario(site=site, 
                         inlet=height, 
                         domain=domain, 
                         species=species, 
                         source=source_waste, 
                         start_date=start_date,
                         end_date=end_date)

/Users/gar/Documents/Devel/env/openenv/lib/python3.9/site-packages/xarray/core/indexes.py:159: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  indexer = index.slice_indexer(


Adding obs_surface to model scenario
Updating any inputs based on observation data
site: tac, species: ch4, inlet: 100m


Adding footprint to model scenario
Adding flux to model scenario
Unable to add boundary_conditions data based on keywords supplied.
 Inputs - 

 species: ch4

 domain: EUROPE

 bc_input: None

 start_date: 2016-07-01

 end_date: 2016-08-01

Filename: /Users/gar/Documents/Devel/openghg/openghg/analyse/_scenario.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    78    177.8 MiB    177.8 MiB           1       @profile
    79                                             def __init__(
    80                                                 self,
    81                                                 site: Optional[str] = None,
    82                                                 species: Optional[str] = None,
    83                                                 inlet: Optional[str] = None,
    84                                                 network: Optional[str] = None,
    85                                                 domain: Optional[str] = None,
    86      

Using these keywords, this will search the object store and attempt to collect and attach observation, footprint and flux data. This collected data will be attached to your created `ModelScenario`. For the observations this will be stored as the `ModelScenario.obs` attribute. This will be an `ObsData` object which contains metadata and data for your observations:

In [ ]:
scenario.obs

To access the undelying xarray Dataset containing the observation data use `ModelScenario.obs.data`:

In [ ]:
ds = scenario.obs.data

The `ModelScenario.footprint` attribute contains the linked FootprintData (again, use `.data` to extract xarray Dataset):

In [ ]:
scenario.footprint

And the `ModelScenario.fluxes` attribute can be used to access the FluxData. Note that for `ModelScenario.fluxes` this can contain multiple flux sources and so this is stored as a dictionary linked to the source name:

In [ ]:
scenario.fluxes

An interactive plot for the linked observation data can be plotted using the `ModelScenario.plot_timeseries()` method:

In [ ]:
scenario.plot_timeseries()

You can also set up your own searches and add this data directly.

In [ ]:
from openghg.retrieve import get_obs_surface, get_footprint, get_flux

# Extract obs results from object store
obs_results = get_obs_surface(site=site,
                              species=species,
                              inlet=height,
                              start_date="2016-07-01",
                              end_date="2016-08-01")

# Extract footprint results from object store
footprint_results = get_footprint(site=site,
                                  domain=domain,
                                  height=height,
                                  start_date="2016-07-01",
                                  end_date="2016-08-01")

# Extract flux results from object store
flux_results = get_flux(species=species,
                        domain=domain,
                        source=source_waste)

In [ ]:
scenario_direct = ModelScenario(obs=obs_results, footprint=footprint_results, flux=flux_results)

*You can create your own input objects directly and add these in the same way. This allows you to bypass the object store for experimental examples. At the moment these inputs need to be `ObsData`, `FootprintData` or `FluxData` objects (can be created using classes from openghg.dataobjects) but simpler inputs will be made available*.

One benefit of this interface is to reduce searching the database if the same data needs to be used for multiple different scenarios.

## 3. Comparing data sources

Once your `ModelScenario` has been created you can then start to use the linked data to compare outputs. For example we may want to calculate modelled observations at our site based on our linkec footprint and emissions data:

In [ ]:
modelled_observations = scenario.calc_modelled_obs()

This could then be plotted directlt using the xarray plotting methods:

In [ ]:
modelled_observations.plot()  # Can plot using xarray plotting methods

To compare the these modelled observations to the ovbservations themselves, the `ModelScenario.plot_comparison()` method can be used.

In [ ]:
scenario.plot_comparison(baseline="percentile")

The `ModelScenario.footprints_data_merge()` method can also be used to created a combined output, with all aligned data stored directly within an `xarray.Dataset`:

In [ ]:
combined_dataset = scenario.footprints_data_merge()
combined_dataset

When the same calculation is being performed for multiple methods, the last calculation is cached to allow the outputs to be produced more efficiently. This can be disabled for large datasets by using `cache=False`.

For a `ModelScenario` object, different analyses can be performed on this linked data. For example if a daily average for the modelled observations was required, we could calculate this setting our `resample_to` input to `"1D"` (matching available pandas time aliases):

In [ ]:
modelled_observations_12H = scenario.calc_modelled_obs(resample_to="1D")
modelled_observations_12H.plot()

To allow comparisons with multiple fluxes inputs, more than one flux source can be linked to your `ModelScenario`. This can be either be done upon creation or can be added using the `add_flux()` method. When calculating modelled observations, these flux sources will be aligned in time and stacked to create a total output:

In [ ]:
scenario.add_flux(species=species, domain=domain, source=source_energyprod)

In [ ]:
scenario.plot_comparison(baseline="percentile")

Output for individual sources can also be created by specifying the `sources` as an input:

In [ ]:
# Included recalculate option to ensure this is updated from cached data.
modelled_obs_energyprod = scenario.calc_modelled_obs(sources="energyprod", recalculate=True)
modelled_obs_energyprod.plot()

*Plotting functions to be added for 2D / 3D data*

---